```python
def gaussian_kernel(x1, x2, sigma):
    return torch.exp(-torch.norm(x1 - x2) ** 2 / (2 * sigma ** 2))
```

O kernel Gaussiano (ou RBF, Radial Basis Function) mede a similaridade entre dois vetores `x1` e `x2`. Ele funciona da seguinte maneira:

- `torch.norm(x1 - x2)`: calcula a distância Euclidiana entre os vetores.
- `sigma`: controla a suavidade da função kernel. Quanto menor o `sigma`, mais "local" é o efeito do kernel.
- `torch.exp(-dist / (2 * sigma ** 2))`: converte a distância em uma medida de similaridade. Quanto mais próximos `x1` e `x2`, maior será o valor retornado pelo kernel, próximo a `1`. Se estiverem distantes, o valor será próximo a `0`.


```python
class RidgeSGDKernelTorch(BaseEstimator, RegressorMixin):
    def __init__(self, eta=0.01, c=1.0, sigma=1.0):
        self.eta = eta
        self.c = c
        self.sigma = sigma
        self.alpha = None
        self.X_train_tensor = None
```

Esta classe implementa um regressor customizado:

- `eta`: taxa de aprendizado usada na atualização dos pesos durante o treinamento (descida de gradiente).
- `c`: parâmetro de regularização para penalizar grandes valores de `alpha`, ajudando a evitar o sobreajuste.
- `sigma`: parâmetro do kernel Gaussiano, controlando o alcance da influência dos pontos.
- `alpha`: vetor de coeficientes que o modelo ajusta online, ou seja, conforme novos dados de treino chegam.
- `X_train_tensor`: armazena os dados de treinamento para calcular o kernel em cada novo exemplo.


```python
def fit(self, X, y):
    self.alpha = None
    self.X_train_tensor = None
    for x_new, y_new in zip(X, y):
        self.partial_fit(x_new, y_new)
    return self

```
O método `fit` percorre todos os pares `(x_new, y_new)` do conjunto de dados e chama `partial_fit` para treinar o modelo de forma incremental. Esse processo ajusta os coeficientes `alpha` com base em cada novo ponto.


```python
def partial_fit(self, x_new, y_new):
    if self.X_train_tensor is None:
        self.X_train_tensor = torch.tensor([x_new], dtype=torch.float32)
        self.alpha = torch.zeros(1, dtype=torch.float32)
    else:
        self.X_train_tensor = torch.vstack([self.X_train_tensor, torch.tensor(x_new, dtype=torch.float32)])
        self.alpha = torch.cat([self.alpha, torch.zeros(1)])

    n_samples = self.X_train_tensor.shape[0]

    if n_samples > 1:
        kernels = torch.tensor([gaussian_kernel(self.X_train_tensor[i], self.X_train_tensor[-1], self.sigma)
                                for i in range(n_samples - 1)])
        pred_n = torch.dot(self.alpha[:n_samples - 1], kernels)
    else:
        pred_n = 0

    error = y_new - pred_n

    self.alpha[:n_samples - 1] = (1 - self.eta * self.c) * self.alpha[:n_samples - 1]
    self.alpha[n_samples - 1] = self.eta * error

```

Este método `partial_fit` realiza um ajuste online (incremental) no modelo, processando um único par `x_new`, `y_new` de cada vez. Aqui está o que cada parte do código faz:

### Armazenamento dos Dados de Treinamento Incremental

- Se `self.X_train_tensor` está vazio (ou seja, é o primeiro ponto a ser processado), inicializamos esse tensor com `x_new` e criamos um vetor `alpha` de zeros, com um valor.
- Se não está vazio, acrescentamos `x_new` a `self.X_train_tensor` e estendemos `alpha` com um novo zero. Dessa forma, todos os dados recebidos são armazenados incrementadamente, um por vez.

### Predição Incremental

- `n_samples` guarda o número total de amostras no tensor de treinamento atual.
- Se há mais de uma amostra, calculamos a similaridade entre `x_new` (última amostra) e cada amostra anterior usando o `gaussian_kernel`.
- `pred_n` é a predição para `x_new`, calculada como a combinação linear das entradas de `alpha` correspondentes às amostras anteriores e seus valores de similaridade (kernels).
- Se há apenas uma amostra, `pred_n` é `0`.

### Erro e Atualização de Alpha

- `error` representa o erro da predição atual: `error = y_new - pred_n`.
- As entradas anteriores de `alpha` são atualizadas para refletir o erro com base na taxa de regularização `c` e taxa de aprendizado `eta`: `(1 - self.eta * self.c) * self.alpha[:n_samples - 1]`.
- A última entrada de `alpha` é atualizada para a proporção do erro atual `self.eta * error`, associando o erro atual ao ponto mais recente, `x_new`.

Assim, esse método ajusta o modelo de forma incremental, acumulando e atualizando os pesos `alpha` com base em cada nova amostra recebida.
